In [1]:
import pandas as pd

orders = pd.read_csv(r"C:\Users\kandr\Downloads\orders.csv")


In [2]:
users = pd.read_json(r"C:\Users\kandr\Downloads\users.json")


In [5]:
import sqlite3
import pandas as pd

# 1. Connect to in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# 2. Load SQL file
sql_path = r"C:\Users\kandr\Downloads\restaurants.sql"

with open(sql_path, "r", encoding="utf-8") as f:
    sql_script = f.read()

# 3. Execute SQL
conn.executescript(sql_script)

# 4. Read table into DataFrame
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print(restaurants.head())


   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [6]:
orders

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian
...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian


In [7]:
users

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


In [8]:
restaurants

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8
...,...,...,...,...
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9


In [9]:
merged1 = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [10]:
final_df = pd.merge(
    merged1,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [11]:
final_df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [13]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)



city
Chennai    1080909.79
Name: total_amount, dtype: float64

In [14]:
final_df.groupby("cuisine")["total_amount"] \
    .mean() \
    .sort_values(ascending=False) \
    .head(1)


cuisine
Mexican    808.021344
Name: total_amount, dtype: float64

In [15]:
user_spend = final_df.groupby("user_id")["total_amount"].sum()

user_count = user_spend[user_spend > 1000].nunique()
user_count


2543

In [16]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)

final_df.groupby("rating_range")["total_amount"] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)


C:\Users\kandr\AppData\Local\Temp\ipykernel_17316\1381325423.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"] \


rating_range
4.6 – 5.0    2197030.75
Name: total_amount, dtype: float64

In [17]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False) \
    .head(1)


city
Chennai    808.45908
Name: total_amount, dtype: float64

In [18]:
final_df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values(["restaurant_count", "total_revenue"], ascending=[True, False])


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [19]:
gold_percentage = round(
    (final_df[final_df["membership"] == "Gold"].shape[0] / final_df.shape[0]) * 100
)

gold_percentage


50

In [20]:
restaurant_stats = final_df.groupby("restaurant_name").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

restaurant_stats[restaurant_stats["total_orders"] < 20] \
    .sort_values("avg_order_value", ascending=False) \
    .head(1)


KeyError: 'restaurant_name'

In [21]:
final_df.groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)


membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64

In [22]:
final_df.groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)


membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64

In [23]:
restaurant_stats = final_df.groupby("restaurant_name").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

restaurant_stats[
    restaurant_stats["total_orders"] < 20
].sort_values(
    "avg_order_value", ascending=False
).head(1)


KeyError: 'restaurant_name'

In [24]:
final_df.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum().sort_values(
    ascending=False
).head(1)


membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64

In [25]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.quarter

final_df.groupby("quarter")["total_amount"] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)


C:\Users\kandr\AppData\Local\Temp\ipykernel_17316\4291040302.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


quarter
3    2037385.1
Name: total_amount, dtype: float64

In [27]:
restaurant_stats = final_df.groupby("restaurant_name_y").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

restaurant_stats[
    restaurant_stats["total_orders"] < 20
].sort_values(
    "avg_order_value", ascending=False
).head(1)


,avg_order_value,total_orders
restaurant_name_y,,
Restaurant_294,1040.222308,13


In [28]:
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]
gold_orders_count


4987

In [29]:
hyderabad_revenue = round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)
hyderabad_revenue


1889367

In [30]:
distinct_users = final_df["user_id"].nunique()
distinct_users


2883

In [31]:
gold_aov = round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)
gold_aov


797.15

In [32]:
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [34]:
top_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .idxmax()
)

top_city
orders_in_top_city = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_city)
].shape[0]

orders_in_top_city


1337

In [35]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           10000 non-null  int64         
 1   user_id            10000 non-null  int64         
 2   restaurant_id      10000 non-null  int64         
 3   order_date         10000 non-null  datetime64[ns]
 4   total_amount       10000 non-null  float64       
 5   restaurant_name_x  10000 non-null  object        
 6   name               10000 non-null  object        
 7   city               10000 non-null  object        
 8   membership         10000 non-null  object        
 9   restaurant_name_y  10000 non-null  object        
 10  cuisine            10000 non-null  object        
 11  rating             10000 non-null  float64       
 12  rating_range       9669 non-null   category      
 13  quarter            10000 non-null  int32         
dtypes: cate